In [2]:
import numpy as np 
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import cross_val_score
# import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from scipy import stats
from lazypredict.Supervised import LazyRegressor

print("done")

done


In [3]:
#read csv
df = pd.read_csv("CH4_ML_dataset.csv")

df.head()

,Date,Latitude,Longitude,Temp,rh,CH4_flux,Chla.x,pH,Time.x,CI,Time.y,Chla.y,PAR,Kd490
0,2024-05-04,43.22,-86.29,12.45,NaN,3.30,0.55,8.03,162159,0.00,162159,0.99,1757.43,0.00
1,2024-05-04,43.22,-86.29,13.80,NaN,2.86,0.70,8.40,162159,0.00,162159,0.99,1757.43,0.00
2,2024-05-04,43.22,-86.29,12.45,NaN,2.74,0.55,8.03,162159,0.00,162159,0.99,1757.43,0.00
3,2024-05-04,43.22,-86.29,13.80,NaN,1.31,0.70,8.40,162159,0.00,162159,0.99,1757.43,0.00
4,2024-05-04,43.22,-86.29,14.70,NaN,0.97,0.70,8.50,162159,0.00,162159,0.99,1757.43,0.00


In [4]:
features = df[['CI', 'Chla.y', 'PAR', 'Kd490']]
feature_names = ['CI', 'Chla.y', 'PAR', 'Kd490']


features = features.to_numpy()
ch4 = df[["CH4_flux"]].to_numpy()

lat = df[["Latitude"]].to_numpy()
lon = df[["Longitude"]].to_numpy()
coordinates = (df.Latitude, df.Longitude)

rs = 621

In [5]:
#%% INITIAL TRAIN-TEST SPLIT (holdout 10% for final training)
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test, lon_train, lon_test, lat_train, lat_test = train_test_split(
    features, ch4, lon, lat, test_size = 0.1, random_state = rs)

In [6]:
#define evaluate function
#%% Define model evaluation function 
#Eval stats selected from Pontius "Metrics that make a difference" book.
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def evaluate(model, test_features, test_labels, mod_name):
    predictions = model.predict(test_features)
    mean_dev = np.mean(predictions) - np.mean(test_labels)
    mae = mean_absolute_error(test_labels, predictions)
    pearson = stats.pearsonr(predictions, test_labels.ravel())[0]
    slope = np.sum(np.multiply((test_labels - test_labels.mean()),\
                            np.reshape(predictions - predictions.mean(),(-1,1))))\
        / np.sum(np.square((test_labels - test_labels.mean())))
    r2 = r2_score(test_labels, predictions)
    rmse = mean_squared_error(test_labels, predictions)
    print('Model Performance')
    print('Mean Deviation: {:0.3f}'.format(np.mean(mean_dev)))
    print('Mean Absolute Error: {:0.3f}'.format(mae))
    print('RMSE: {:0.3f}'.format(rmse))
    print('Slope = {:0.3f}'.format(slope))
    print('Pearson correlation = {:0.3f}'.format(pearson))
    print('r2 = {:0.3f}'.format(r2))
    values = [mean_dev, mae, rmse, slope, pearson, r2]
    metrics = pd.DataFrame(values, 
                           columns = ['statistic'])
    metrics.index = ['MeanDeviation','MAE','RMSE', 'Slope',
               'Pearson', 'r2']

In [7]:
#lazy regressor analysis
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(x_train, x_test, y_train, y_test)

print(models)

 26%|██▌       | 11/42 [00:01<00:03,  8.51it/s]

GammaRegressor model failed to execute
Some value(s) of y are out of the valid range of the loss 'HalfGammaLoss'.


 71%|███████▏  | 30/42 [00:04<00:01,  7.52it/s]

PoissonRegressor model failed to execute
Some value(s) of y are out of the valid range of the loss 'HalfPoissonLoss'.


100%|██████████| 42/42 [00:08<00:00,  4.94it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002378 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 137
[LightGBM] [Info] Number of data points in the train set: 463, number of used features: 4
[LightGBM] [Info] Start training from score 31.748369
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i